# Basic Submission

This Jupyter Notebook is designed to explore a variety of advanced optimization techniques applied to two specific functions: the well-known Rosenbrock function and a custom-designed function. This study encompasses Newton-based techniques, conjugate gradient techniques if the linear and non-linear type, quasi-Newton methods and gradient and Hessian approximation. The aim is to comprehensively compare the effectiveness and efficiency of these diverse approaches in finding global minima and understanding their behavior under different mathematical conditions and complexities by subjecting them to a representative test suite.


## Coverage

For each of three starting points starting points:

| Problem Domain          | Rosenbrock | Custom Function | Derivative Approximation (T5) | Comment                       |
|-------------------------|-----|----------|------------------------|-------------------------------|
| Newton (T1)             |✅|✅|✅|                               |
| Linear CG (T2)          |✅|✅|✅|                               |
| Non-Linear CG: FR (T3)  |✅|✅|✅|                               |
| Non-Linear CG: PR (T3)  |✅|✅|✅|                               |
| Quasi-Newton: BFGS (T4) |✅|✅|✅|                               |
| Quasi-Newton: SR1 (T4)  |✅|✅|✅| Trust Region FW _not_ covered |
| Outperform Newton (T6)  |✅|✅|(n/a)|                               |

## Theoretical Background

### Target Functions


#### Rosenbrock Function

The Rosenbrock function is formulated as:
$$ f(x) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2 $$

To find the global minimum analytically, we start by setting the gradient of the function to zero. The gradient components are:
$$ \frac{\partial f}{\partial x_1} = -400x_1(x_2 - x_1^2) - 2(1 - x_1) $$
$$ \frac{\partial f}{\partial x_2} = 200(x_2 - x_1^2) $$

Setting these derivatives to zero, we derive:

1. From $\frac{\partial f}{\partial x_2} = 0$, we find that $x_2 = x_1^2$.
2. Substituting $x_2 = x_1^2$ into $\frac{\partial f}{\partial x_1}$ and simplifying, we get:
   $$ -400x_1(x_1^2 - x_1^2) - 2(1 - x_1) = 0 $$
   $$ -2(1 - x_1) = 0 $$
   Leading to $x_1 = 1$.

Given $x_1 = 1$, and substituting back, we find $x_2 = 1^2 = 1$.

Therefore, the global minimum is at $(x_1, x_2) = (1, 1)$ where $f(x) = 0$, lying at the bottom of a long, narrow, parabolic-shaped valley.


In [32]:
import numpy as np

def safe_rosenbrock(x):
    """Calculate the Rosenbrock function value at x."""
    # Clipping x values to prevent excessive values
    x = np.clip(x, -10, 10)
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2


In [33]:
import numpy as np

def safe_grad_rosenbrock(x):
    """Calculate the gradient of the Rosenbrock function at x."""
    x = np.clip(x, -10, 10)
    grad = np.zeros(2)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[1] = 200 * (x[1] - x[0]**2)
    return grad


In [34]:
def hessian_rosenbrock(x):
    """ Calculate the Hessian matrix of the Rosenbrock function at x. """
    x1, x2 = x
    return np.array([[1200 * x1**2 - 400 * x2 + 2, -400 * x1],
                     [-400 * x1, 200]])


#### Custom Function

The custom function is defined as:
$$ f(x) = 150(x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2 $$

To analyze the global minimum, we set the gradient to zero. The gradient components are:
$$ \frac{\partial f}{\partial x_1} = 300 x_1 x_2^2 + (0.5 x_1 + 2 x_2 - 2) $$
$$ \frac{\partial f}{\partial x_2} = 300 x_1^2 x_2 + 4(0.5 x_1 + 2 x_2 - 2) $$

Setting these derivatives to zero, we simplify and solve:

1. From $\frac{\partial f}{\partial x_1} = 0$ and $\frac{\partial f}{\partial x_2} = 0$, assuming $x_1 = 0$, we find:
   $$ 2 x_2 - 2 = 0 $$
   Leading to $x_2 = 1$.
2. Substituting $x_1 = 0$ into $\frac{\partial f}{\partial x_2}$ yields:
   $$ 4(2 x_2 - 2) = 0 $$
   Confirms that $x_2 = 1$.

Therefore, the point $(x_1, x_2) = (0, 1)$ represents a critical point. Further analysis, potentially including second derivative tests or numerical verification, would confirm its nature as a global minimum.


To determine if there is another point where $f(x_1, x_2) = 0$ for the custom function:
$$ f(x) = 150(x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2 $$

we need to find conditions under which both terms in the function evaluate to zero because this is the only way their sum can be zero. Here's the breakdown:

**Analyzing Each Term for Zero:**

1. **The first term $150(x_1 x_2)^2$ equals zero when:**
   - $x_1 = 0$ or
   - $x_2 = 0$

2. **The second term $(0.5 x_1 + 2 x_2 - 2)^2$ equals zero when:**
   - $0.5 x_1 + 2 x_2 - 2 = 0$

**Solving for Conditions:**

Given the conditions from the second term, we rearrange the equation:
$$ 0.5 x_1 + 2 x_2 = 2 $$
$$ x_2 = 1 - 0.25 x_1 $$

Now, substitute this expression for $x_2$ into the condition from the first term:
$$ x_1 \cdot (1 - 0.25 x_1) = 0 $$

This equation is satisfied when $x_1 = 0$ or $x_1 = 4$. Let's explore both:

- **If $x_1 = 0$**:
  - Substituting $x_1 = 0$ in $x_2 = 1 - 0.25 \cdot 0$:
  - $x_2 = 1$
  - This point $(0, 1)$ was already identified as a global minimizer where $f(x) = 0$.

- **If $x_1 = 4$**:
  - Substituting $x_1 = 4$ in $x_2 = 1 - 0.25 \cdot 4$:
  - $x_2 = 0$
  - This results in the point $(4, 0)$, and checking $f(4, 0)$:
  - $f(4, 0) = 150(4 \cdot 0)^2 + (0.5 \cdot 4 + 2 \cdot 0 - 2)^2 = 0$
  - This shows $f(4, 0)$ is zero, too, constituting a second global minimizer candidate.



To verify whether $(4, 0)$ is also a minimum, we evaluate the gradient components at this point.

At $(4, 0)$ we get:

$$ \frac{\partial f}{\partial x_1} = 300 \cdot 4 \cdot 0^2 + 0.5(0.5 \cdot 4 + 2 \cdot 0 - 2) = 0 $$
$$ \frac{\partial f}{\partial x_2} = 300 \cdot 4^2 \cdot 0 + 2(0.5 \cdot 4 + 2 \cdot 0 - 2) = 0 $$

Both derivatives are zero, confirming that $(4, 0)$ is a critical point. As with $(0,1)$, further analysis would confirm its nature as a minimum.


In [35]:
import numpy as np

def custom_function(x):
    """Calculate the custom function value at x."""
    return 150 * (x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2


In [36]:
import numpy as np

def grad_custom_function(x):
    """Calculate the gradient of the custom function at x."""
    grad = np.zeros(2)
    grad[0] = 300 * x[0] * x[1]**2 + 2 * (0.5 * x[0] + 2 * x[1] - 2) * 0.5
    grad[1] = 300 * x[0]**2 * x[1] + 2 * (0.5 * x[0] + 2 * x[1] - 2) * 2
    return grad


In [37]:
def hessian_custom_function(x):
    """Calculate the Hessian matrix of the custom function at x. """
    x1, x2 = x
    return np.array([[300 * x2**2 + 0.5, 600 * x1 * x2 + 2],
                     [600 * x1 * x2 + 2, 300 * x1**2 + 8]])

## Methodological Approach

### Line Search Methods

A crucial component in optimization algorithms that use gradient information is the line search technique. The line search aims to find an acceptable step size that satisfies certain conditions, improving the convergence of the method.

#### Backtracking Line Search

**Description:**
Backtracking line search is an iterative method for determining the step size in optimization algorithms. This method reduces the step size until a decrease in the function value satisfies the Armijo condition, a form of the Wolfe conditions, which are used to ensure sufficient decrease and convergence.

**Mathematical Formulation:**
Backtracking line search involves iteratively reducing the step size $\alpha$ based on a shrinkage factor until the function value at the new point is less than the value at the current point decreased by a scaled gradient magnitude. The condition is formally given by:

$$ f(x + \alpha p) \leq f(x) + c \alpha \nabla f(x)^T p $$

where:
- $\alpha$ is the step size,
- $p$ is the search direction (typically the negative gradient),
- $c$ is a small scalar, usually between $0.01$ and $0.3$,
- $\nabla f(x)$ is the gradient of the function at $x$.

**Algorithmic Steps:**
1. Start with an initial guess for the step size, usually $\alpha = 1$.
2. Evaluate the function at $x + \alpha p$.
3. If the function value at $x + \alpha p$ is more than $f(x) + c \alpha \nabla f(x)^T p$, reduce $\alpha$ by multiplying it by a reduction factor $\rho$ (typically around $0.5$ to $0.8$).
4. Repeat the evaluation until the condition is met.
5. Once the condition is satisfied, accept $\alpha$ as the step size.

**Rationale and Use Case:**
Backtracking line search is particularly useful in non-convex optimization problems where choosing an appropriate step size is crucial for convergence. It adapts the step size based on the local landscape of the function, helping to avoid steps that are too large which may lead to divergence or overshooting the minimum.

**Heuristics and Parameter Choices:**
- The initial step size and the values of $c$ and $\rho$ are crucial parameters. $c$ should be small to ensure sufficient decrease, while $\rho$ should be chosen to make meaningful reductions in step size.
- It's important to balance the aggressiveness of the reduction to avoid excessive function evaluations.

**Advantages and Limitations:**
- **Advantages:** Simple to implement and does not require any complex calculations beyond function evaluation and gradient computation. It is robust and can be used in conjunction with a variety of optimization algorithms.
- **Limitations:** Can result in a large number of function evaluations, especially if the initial guess for $\alpha$ is far from optimal. The method might also make very slow progress if the parameters are not chosen well.


In [38]:
def line_search(func, grad_func, x, d, alpha_init=1.0, rho=0.9, c=1e-4, max_iter=50):
    """
    Conducts a backtracking line search to find the step size that satisfies the Armijo condition.
    
    This line search method reduces the step size alpha iteratively until a decrease in the function
    value satisfies the Armijo condition, which ensures sufficient decrease.

    Args:
        func (callable): The objective function to minimize. It should take a single numpy array argument.
        grad_func (callable): The gradient of the objective function. It should take a single numpy array argument.
        x (np.array): The current point in the parameter space where the function is evaluated.
        d (np.array): The current search direction along which the line search is performed.
        alpha_init (float): The initial step size for the line search.
        rho (float): The factor by which the step size is reduced in each iteration (0 < rho < 1).
        c (float): The Armijo constant used in the sufficient decrease condition (0 < c < 1).
        max_iter (int): The maximum number of iterations to perform if the Armijo condition is not met.

    Returns:
        tuple: A tuple containing:
            - alpha (float): The step size that satisfies the Armijo condition or the step size at the end of max_iter iterations.
            - log (list of dicts): A log of each iteration's details including the iteration number, alpha value, function value, and target Armijo condition value.
    """
    alpha = alpha_init
    log = []
    iteration = 0
    while True:
        f_current = func(x)
        grad_current = grad_func(x)
        f_test = func(x + alpha * d)
        armijo_condition = f_current + c * alpha * np.dot(grad_current, d)
        log.append({
            'iteration': iteration + 1,
            'alpha': alpha,
            'function_value': f_test,
            'target_value': armijo_condition
        })
#        if f_test <= armijo_condition or iteration >= max_iter:
#            break
        if f_test <= armijo_condition:
            break
        alpha *= rho
        iteration += 1
    return alpha, log


### Optimization Algorithms: Newton Method with Hessian Modification

**Description:**
Newton's method is a second-order optimization algorithm that uses the Hessian matrix of second derivatives to find the zeros of a function, or the local maxima and minima. This version includes a modification to ensure the Hessian matrix is always positive definite, enhancing stability and convergence.

**Mathematical Formulation:**
Given an objective function $f(x)$, Newton's method updates the current point $x$ according to:

$$ x_{\text{new}} = x_{\text{old}} - H^{-1}(x_{\text{old}}) \nabla f(x_{\text{old}}) $$

where $H(x)$ is the Hessian matrix and $\nabla f(x)$ is the gradient. If $H(x)$ has non-positive eigenvalues, we adjust it by adding $\lambda I$ where $\lambda$ is slightly larger than $|\text{min eigenvalue}|$.

**Algorithmic Steps:**
1. Compute the gradient $\nabla f(x)$ and the Hessian $H(x)$ at the current point.
2. Correct $H(x)$ to ensure it is positive definite.
3. Compute the inverse of the corrected Hessian.
4. Update the point $x$ using the formula above.
5. Check for convergence. If the change in $x$ or in $f(x)$ is below a threshold, stop; otherwise, repeat.

**Rationale:**
This method is particularly effective for functions that are well-approximated by a quadratic form near the optimum, due to its use of second-order information. The eigenvalue correction ensures that the update direction is always a descent direction, preventing divergences.

**Heuristics and Parameter Choices:**
- The convergence threshold and the size of the eigenvalue correction can significantly affect the performance and stability of the method. These need to be chosen based on the scale and sensitivity of the objective function.

**Advantages and Limitations:**
- **Advantages:** Fast convergence near the optimum when the function is quadratic.
- **Limitations:** High computational cost from calculating and inverting the Hessian; potential numerical instability without modifications.


In [39]:
import numpy as np

def newton_eigen_method(func, gradient, hessian, x0, known_solution, max_iter=1000, tol=1e-6):
    """
    Newton method with eigenvalue correction to ensure the Hessian is positive definite.

    Args:
        func (callable): The objective function to minimize.
        gradient (callable): The gradient of the function.
        hessian (callable): The Hessian of the function.
        x0 (np.array): Initial guess for the parameters.
        known_solution (np.array): Known solution for calculating the distance.
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance.

    Returns:
        np.array: The optimized parameters at convergence.
        int: Total number of iterations performed.
        list: Detailed log of the optimization process.
        float: Norm of the final gradient.
        float: Distance to the known solution.
    """
    x = np.array(x0, dtype=float)
    overall_log = []
    for k in range(max_iter):
        grad = gradient(x)
        hess = hessian(x)

        # Eigenvalue correction if not positive definite
        eigvals, _ = np.linalg.eig(hess)
        min_eigval = np.min(eigvals)
        if min_eigval <= 0:
            hess += np.eye(len(x)) * (-min_eigval + 1e-6)

        pk = -np.linalg.solve(hess, grad)
        # Ensure your line_search function handles this and returns alpha and ls_log
        alpha, ls_log = line_search(func, gradient, x, pk)  
        
        x_new = x + alpha * pk
        if np.linalg.norm(grad) < tol:
            break

        overall_log.append({
            'iteration': k + 1,
            'x': x_new.copy(),
            'gradient_norm': np.linalg.norm(grad),
            'function_value': func(x_new),
            'distance_to_solution': np.linalg.norm(x_new - known_solution),
            'line_search_log': ls_log
        })
        x = x_new

    final_gradient_norm = np.linalg.norm(gradient(x))
    distance_to_solution = np.linalg.norm(x - known_solution)

    return x, k + 1, overall_log, final_gradient_norm, distance_to_solution


In [40]:
import unittest

class TestNewtonEigenMethod(unittest.TestCase):
    def test_newton_eigen_convergence(self):
        # Define a quadratic function for testing
        def func(x):
            return 0.5 * (x[0]**2 + 10 * x[1]**2)

        def gradient(x):
            return np.array([x[0], 10 * x[1]])

        def hessian(x):
            return np.array([[1, 0], [0, 10]])

        x0 = np.array([10.0, -10.0])
        known_solution = np.array([0.0, 0.0])

        optimized_x, num_iters, log, _, _ = newton_eigen_method(
            func, gradient, hessian, x0, known_solution, max_iter=100, tol=1e-6
        )

        # Check if solution converges to the known solution
        np.testing.assert_allclose(optimized_x, known_solution, atol=1e-6, rtol=1e-6)
        self.assertTrue(len(log) > 0, "No logging information provided.")
        print("Optimization details:", log[-1])

test_suite = unittest.TestSuite()
loader = unittest.TestLoader()
test_suite.addTest(loader.loadTestsFromTestCase(TestNewtonEigenMethod))
runner = unittest.TextTestRunner()
runner.run(test_suite)


.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Optimization details: {'iteration': 1, 'x': array([0., 0.]), 'gradient_norm': 100.4987562112089, 'function_value': 0.0, 'distance_to_solution': 0.0, 'line_search_log': [{'iteration': 1, 'alpha': 1.0, 'function_value': 0.0, 'target_value': 549.89}]}


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Optimization Algorithms: Conjugate Gradient Methods

With the line search method defined, we can now incorporate it into the conjugate gradient algorithm, an efficient method for solving large-scale optimization problems. The conjugate gradient method uses line search to determine the optimal step size in each iteration, enhancing the algorithm's overall effectiveness.

Conjugate gradient methods are iterative techniques that build a sequence of conjugate directions, along which the function is minimized. The general approach involves computing a search direction that is a linear combination of the steepest descent direction and the previous search direction.

### Linear Conjugate Gradient (LCG)

**Description:**
The Linear Conjugate Gradient Method is an algorithm for solving systems of linear equations with a positive-definite matrix, or for solving linear optimization problems. In optimization, it is used for minimizing quadratic functions in the form $f(x) = \frac{1}{2} x^T A x - b^T x$, but it can be generalized for non-quadratic functions using appropriate line search techniques.

**Mathematical Formulation:**
The objective function optimized by LCG is typically a quadratic function:

$$ f(x) = \frac{1}{2} x^T A x - b^T x + c $$

The update rules for the conjugate gradient method in the context of optimization are:

1. **Initialization:**
   - Choose an initial guess $x_0$.
   - Set initial gradient $g_0 = \nabla f(x_0)$ and initial direction $d_0 = -g_0$.

2. **Iterative Update:**
   - For each iteration $k$, compute:
     - Step size $\alpha_k = \frac{g_k^T g_k}{d_k^T A d_k}$.
     - Update position $x_{k+1} = x_k + \alpha_k d_k$.
     - Update gradient $g_{k+1} = \nabla f(x_{k+1})$.
     - Compute $\beta_{k+1} = \frac{g_{k+1}^T g_{k+1}}{g_k^T g_k}$.
     - Update direction $d_{k+1} = -g_{k+1} + \beta_{k+1} d_k$.

**Algorithmic Steps:**
1. Compute initial gradient and set initial direction.
2. For each iteration, perform a line search to determine the optimal step size.
3. Update the position and gradient based on the line search result.
4. Adjust the search direction using the computed $\beta$ value.
5. Check for convergence. If the norm of the gradient is below a threshold, terminate.

**Rationale and Use Case:**
LCG is particularly effective for high-dimensional optimization problems where the Hessian matrix is sparse and computing or storing it explicitly is impractical. It requires only gradient evaluations and is more memory efficient compared to methods like BFGS or Newton's method.

**Heuristics and Parameter Choices:**
- The tolerance for convergence (`tol`) and maximum iterations (`max_iter`) are crucial for preventing excessive computations and ensuring timely termination of the algorithm.
- The line search method must be robust to ensure that $\alpha_k$ appropriately minimizes along the direction $d_k$.

**Advantages and Limitations:**
- **Advantages:** Efficient for large sparse systems; only requires gradient information and uses minimal memory.
- **Limitations:** Convergence can be slow near the optimum; performance heavily depends on the line search quality and may stagnate in non-quadratic landscapes.



In [41]:
def linear_conjugate_gradient(func, grad, x0, known_solution, max_iter=1000, tol=1e-6):
    x = np.array(x0)
    g = grad(x)
    d = -g
    overall_log = []
    for k in range(max_iter):
        alpha_k, _ = line_search(func, grad, x, d)  # Ensure you have a line_search function
        if alpha_k is None:
            break
        x_new = x + alpha_k * d
        g_new = grad(x_new)
        beta_k = np.dot(g_new, g_new) / np.dot(g, g)
        d = -g_new + beta_k * d
        g = g_new
        x = x_new
        overall_log.append({
            'iteration': k + 1,
            'x': x.copy(),
            'gradient_norm': np.linalg.norm(g),
            'function_value': func(x),
            'distance_to_solution': np.linalg.norm(x - known_solution)
        })
        if np.linalg.norm(g) < tol:
            break
    return x, k, overall_log, np.linalg.norm(g), np.linalg.norm(x - known_solution)


In [42]:
import unittest

class TestLinearConjugateGradient(unittest.TestCase):
    def test_linear_conjugate_gradient_multi_step(self):
        # Define a simple quadratic function and its gradient
        def func(x):
            return (x[0] - 1)**2 + (x[1] - 2)**2
        
        def grad(x):
            return np.array([2*(x[0] - 1), 2*(x[1] - 2)])

        x0 = np.array([0.0, 0.0])
        known_solution = np.array([1, 2])
        x_final, num_iters, log, final_grad_norm, distance = linear_conjugate_gradient(
            func, grad, x0, known_solution, max_iter=100, tol=1e-6
        )

        # Assert final position is close to known solution
        np.testing.assert_allclose(x_final, known_solution, atol=1e-4)
        # Assert the number of iterations is reasonable
        self.assertTrue(num_iters > 0 and num_iters < 100)
        # Assert the final gradient norm is small, indicating convergence
        self.assertLess(final_grad_norm, 1e-6)
        # Check detailed logs have been created
        self.assertTrue(len(log) > 0)
        # Optional: print some logs for manual inspection
        for entry in log[:10]:  # Print first 10 entries to check
            print(entry)

# Run the tests
test_suite = unittest.TestSuite()
loader = unittest.TestLoader()
test_suite.addTest(loader.loadTestsFromTestCase(TestLinearConjugateGradient))
runner = unittest.TextTestRunner()
runner.run(test_suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.025s

OK


{'iteration': 1, 'x': array([1.8, 3.6]), 'gradient_norm': 3.577708763999664, 'function_value': 3.2000000000000006, 'distance_to_solution': 1.788854381999832}
{'iteration': 2, 'x': array([1.48, 2.96]), 'gradient_norm': 2.146625258399799, 'function_value': 1.152000000000001, 'distance_to_solution': 1.0733126291998996}
{'iteration': 3, 'x': array([0.609088, 1.218176]), 'gradient_norm': 1.7482116104407963, 'function_value': 0.7640609587200007, 'distance_to_solution': 0.8741058052203982}
{'iteration': 4, 'x': array([0.67778844, 1.35557688]), 'gradient_norm': 1.4409739091754885, 'function_value': 0.5191014517311222, 'distance_to_solution': 0.7204869545877443}
{'iteration': 5, 'x': array([1.29977669, 2.59955339]), 'gradient_norm': 1.3406421262802704, 'function_value': 0.44933032768932113, 'distance_to_solution': 0.6703210631401352}
{'iteration': 6, 'x': array([1.29843261, 2.59686522]), 'gradient_norm': 1.3346312085886298, 'function_value': 0.4453101157346867, 'distance_to_solution': 0.6673156

<unittest.runner.TextTestResult run=1 errors=0 failures=0>

#### Fletcher-Reeves Method (FR)

**Description:**
The Fletcher-Reeves (FR) method is a conjugate gradient technique used for nonlinear optimization. It iteratively minimizes multivariable functions by combining the concepts of steepest descent and conjugate directions, without requiring the computation of the Hessian matrix.

**Mathematical Formulation:**
The method is used to minimize a function $f(x)$, where $x$ is a vector in $\mathbb{R}^n$. It updates the variables by iteratively moving towards a minimum, using gradients and a series of conjugate vectors.

1. **Initialization:**
   - Start with an initial guess $x_0$.
   - Compute the initial gradient $g_0 = \nabla f(x_0)$.
   - Set the initial direction $d_0 = -g_0$.

2. **Iterative Update:**
   - For each iteration $k$, perform the following steps:
     - Determine the step size $\alpha_k$ using a line search to minimize $f(x_k + \alpha_k d_k)$.
     - Update the position: $x_{k+1} = x_k + \alpha_k d_k$.
     - Compute the new gradient: $g_{k+1} = \nabla f(x_{k+1})$.
     - Update the direction using the Fletcher-Reeves formula: $\beta_{k+1} = \frac{\|g_{k+1}\|^2}{\|g_k\|^2}$.
     - Set the new direction: $d_{k+1} = -g_{k+1} + \beta_{k+1} d_k$.

**Algorithmic Steps:**
1. Initialize gradient and direction based on the first evaluation of the function's gradient.
2. Use a line search to find the optimal step size that minimizes the function along the current direction.
3. Update the position and gradient.
4. Calculate the parameter $\beta$ that defines the next conjugate direction.
5. Repeat the process until convergence is achieved, typically when the gradient's norm falls below a specified tolerance.

**Rationale and Use Case:**
The Fletcher-Reeves method is particularly useful for functions where the Hessian matrix is difficult to compute or where the dimensionality is so high that other methods become computationally infeasible. It is suitable for smooth nonlinear optimization problems and is favored for its robustness and relatively simple implementation.

**Heuristics and Parameter Choices:**
- Proper selection of the line search method is crucial for the performance of the Fletcher-Reeves method.
- The tolerance for the gradient's norm and the maximum number of iterations are key parameters that control the termination of the algorithm.

**Advantages and Limitations:**
- **Advantages:** Does not require the computation of the Hessian matrix, reducing memory and computational overhead.
- **Limitations:** The performance can be highly sensitive to the line search quality. Poor line search strategies can lead to suboptimal convergence rates or convergence to non-optimal points. Additionally, it may exhibit slow convergence rates near saddle points or minimizers.


#### Polak-Ribiere Method (PR)

**Description:**
The Polak-Ribiere (PR) method is an optimization algorithm that enhances the basic conjugate gradient approach by modifying the formula used to calculate the conjugate direction. It is particularly effective for non-quadratic, nonlinear optimization problems.

**Mathematical Formulation:**
The PR method aims to minimize a function $f(x)$, where $x$ is a vector in $\mathbb{R}^n$. The updates are calculated by:
1. **Initialization:**
   - Choose an initial guess $x_0$.
   - Compute the initial gradient $g_0 = \nabla f(x_0)$.
   - Set the initial direction $d_0 = -g_0$.

2. **Iterative Update:**
   - For each iteration $k$, perform the following steps:
     - Determine the step size $\alpha_k$ through a line search to minimize $f(x_k + \alpha_k d_k)$.
     - Update the position: $x_{k+1} = x_k + \alpha_k d_k$.
     - Compute the new gradient: $g_{k+1} = \nabla f(x_{k+1})$.
     - Update the direction using the Polak-Ribiere formula: $\beta_{k+1} = \frac{(g_{k+1} - g_k)^T g_{k+1}}{\|g_k\|^2}$.
     - Set the new direction: $d_{k+1} = -g_{k+1} + \beta_{k+1} d_k$.

**Algorithmic Steps:**
1. Initialize gradient and direction based on the first evaluation of the function’s gradient.
2. Use a line search to find the optimal step size that minimizes the function along the current direction.
3. Update the position and gradient.
4. Calculate the parameter $\beta$ that modifies the next conjugate direction.
5. Repeat the process until convergence is achieved, typically when the gradient’s norm falls below a specified tolerance.

**Rationale and Use Case:**
The Polak-Ribiere method adjusts the conjugate direction more effectively than the Fletcher-Reeves method, especially in cases where the function landscape is complex and non-quadratic. This leads to potentially faster convergence and improved robustness against the pitfalls of simpler gradient-based methods.

**Heuristics and Parameter Choices:**
- Selection of the line search method is critical for ensuring good performance of the PR method.
- Tolerance levels for stopping criteria and the maximum number of iterations need careful adjustment based on the problem characteristics.

**Advantages and Limitations:**
- **Advantages:** Often achieves faster convergence than the Fletcher-Reeves method, particularly on non-quadratic functions. Requires only first derivatives.
- **Limitations:** Like other conjugate gradient methods, its performance heavily depends on an effective line search strategy. It can be less effective if the underlying function is not well-behaved or if the gradient calculations are inaccurate.




In [43]:
import numpy as np

def conjugate_gradient_method(func, grad_func, x0, known_solution, method='FR', max_iter=10000, tol=1e-6, alpha_init=1.0, rho=0.9, c=1e-4, ls_max_iter=50):
    """
    Implements the conjugate gradient optimization algorithm using either Fletcher-Reeves or Polak-Ribiere update rules, with backtracking line search.

    Args:
        func (callable): The objective function to minimize.
        grad_func (callable): The gradient of the objective function.
        x0 (np.array): Initial guess for the parameters.
        known_solution (np.array): The known solution or global minimum for the function, used for calculating distance.
        method (str): 'FR' for Fletcher-Reeves or 'PR' for Polak-Ribiere.
        max_iter (int): Maximum number of iterations before termination.
        tol (float): Tolerance for convergence, based on the norm of the gradient.
        alpha_init (float): Initial step size for the line search.
        rho (float): Contraction factor in the line search, typically between 0.1 and 0.9.
        c (float): The Armijo rule constant in the line search.
        ls_max_iter (int): Maximum number of iterations allowed in the line search.

    Returns:
        tuple: A tuple containing:
               - Final parameter values (np.array)
               - Number of iterations performed (int)
               - Detailed log of the optimization process (list of dicts)
               - Final gradient norm (float)
               - Distance to the known solution (float)
    """
    x = x0
    g = grad_func(x)
    d = -g
    overall_log = []
    for i in range(max_iter):
        alpha, ls_log = line_search(func, grad_func, x, d, alpha_init, rho, c, ls_max_iter)
        x_new = x + alpha * d
        g_new = grad_func(x_new)
        
        beta = np.dot(g_new, g_new) / np.dot(g, g) if method == 'FR' else np.dot(g_new, g_new - g) / np.dot(g, g)
        
        d = -g_new + beta * d
        if np.linalg.norm(g_new) < tol:
            break
        x = x_new
        g = g_new
        
        overall_log.append({
            'iteration': i + 1,
            'x': x.copy(),
            'gradient_norm': np.linalg.norm(g),
            'alpha': alpha,
            'beta': beta,
            'function_value': func(x),
            'line_search_log': ls_log
        })

    final_gradient_norm = np.linalg.norm(g_new)
    distance_to_solution = np.linalg.norm(x - known_solution)

    return x, i, overall_log, final_gradient_norm, distance_to_solution


### Optimization Algorithms: Quasi-Newton Methods

#### Broyden-Fletcher-Goldfarb-Shanno (BFGS) Method

**Description:**
The BFGS method is a quasi-Newton technique for finding local maxima and minima of functions. Unlike true Newton methods which require the computation of the Hessian matrix, BFGS uses an approximation to the Hessian matrix which is updated iteratively at each step of the optimization process.

**Mathematical Formulation:**
The BFGS method aims to solve the optimization problem for a function $f(x)$ where $x$ is a vector in $\mathbb{R}^n$. The algorithm updates the approximation to the inverse Hessian matrix $H_k$ and the position vector $x_k$ iteratively as follows:

1. **Initialization:**
   - Start with an initial guess $x_0$.
   - Initialize the inverse Hessian approximation, $H_0$, typically to the identity matrix.

2. **Iterative Update:**
   - For each iteration $k$, perform the following steps:
     - Compute the gradient: $g_k = \nabla f(x_k)$.
     - Compute the search direction: $p_k = -H_k g_k$.
     - Determine the step size $\alpha_k$ using a line search to minimize $f(x_k + \alpha_k p_k)$.
     - Update the position: $x_{k+1} = x_k + \alpha_k p_k$.
     - Update the gradient: $g_{k+1} = \nabla f(x_{k+1})$.
     - Compute the difference vectors: $s_k = x_{k+1} - x_k$ and $y_k = g_{k+1} - g_k$.
     - Update the inverse Hessian approximation using the BFGS formula:
       $$ H_{k+1} = (I - \rho_k s_k y_k^T) H_k (I - \rho_k y_k s_k^T) + \rho_k s_k s_k^T $$
       where $\rho_k = \frac{1}{y_k^T s_k}$.

**Algorithmic Steps:**
1. Calculate the gradient and determine the search direction.
2. Use a line search to compute the step size.
3. Update the position and gradient.
4. Adjust the inverse Hessian approximation.
5. Repeat the process until the norm of the gradient is less than a specified tolerance, indicating convergence.

**Rationale and Use Case:**
The BFGS method is used extensively in applications requiring optimization without explicit second derivatives. It is robust, reliable, and often performs well in practical situations.

**Heuristics and Parameter Choices:**
- The initial approximation of the inverse Hessian ($H_0$) is crucial. While the identity matrix is a common choice, other problem-specific initializations can enhance performance.
- The choice of line search method can significantly affect the convergence properties of the BFGS method.

**Advantages and Limitations:**
- **Advantages:** Does not require the Hessian matrix, only first derivatives. Typically has superlinear convergence and is efficient for a wide range of problems.
- **Limitations:** The memory requirement can be substantial for large-dimensional problems due to the need to store the inverse Hessian approximation. The performance can degrade if the line search is not performed adequately.


In [44]:
def bfgs_update(Hk, sk, yk, quasi_zero=1e-12):
    """ Perform the BFGS update on the approximate Hessian (or its inverse).
    
    Args:
        Hk (np.ndarray): Current Hessian approximation.
        sk (np.ndarray): Step vector x_k+1 - x_k.
        yk (np.ndarray): Change in gradients grad f(x_k+1) - grad f(x_k).
    
    Returns:
        np.ndarray: Updated Hessian approximation.
    """
    ykTsk = np.dot(yk.T, sk)
    if np.abs(ykTsk) > quasi_zero:
        rho_k = 1 / ykTsk
        return Hk + rho_k * np.outer(sk, sk) - np.dot(Hk, np.outer(yk, yk)).dot(Hk) / np.dot(yk.T, Hk.dot(yk))
    else:
        return Hk


In [45]:
import numpy as np
import unittest

# Define the BFGS update function
#def bfgs_update_H(H, s, y):
#    rho = 1 / np.dot(y, s)
#    I = np.eye(len(s))
#    V = np.outer(s, y)
#    H_next = (I - rho * V) @ H @ (I - rho * V.T) + rho * np.outer(s, s)
#    return H_next
def bfgs_update_B(Bk, sk, yk, quasi_zero=1e-12):
    ykTsk = np.dot(yk.T, sk)
    if np.abs(ykTsk) > quasi_zero:
        rho_k = 1 / np.dot(yk.T, sk)
        term1 = np.eye(len(Bk)) - rho_k * np.outer(yk, sk)
        term2 = np.eye(len(Bk)) - rho_k * np.outer(sk, yk)
        return np.dot(term1, np.dot(Bk, term2)) + rho_k * np.outer(yk, yk)
    else:
        return Bk

def bfgs_update_H(Hk, sk, yk, quasi_zero=1e-12):
    ykTsk = np.dot(yk.T, sk)
    if ykTsk > quasi_zero:
        rho_k = 1 / np.dot(yk.T, sk)
        return Hk + rho_k * np.outer(sk, sk) - np.dot(Hk, np.outer(yk, yk)).dot(Hk) / np.dot(yk.T, Hk.dot(yk))
    else:
        return Hk

# Create a test class
class TestBFGSHessianAndInverseUpdate(unittest.TestCase):
    def test_bfgs_Hk_update_multi_step(self):
        # Define parameters for a simple quadratic function f(x) = 1/2 x^T Q x - b^T x
        Q = np.array([[4, 1], [1, 3]])  # True Hessian
        Q_inv = np.linalg.inv(Q)  # True inverse Hessian
        b = np.array([1, 1])
        x = np.array([1.0, 1.0])  # Initial point
        grad_f = lambda x: np.dot(Q, x) - b
        
        H = np.eye(2)  # Start with identity matrix as initial inverse Hessian approximation
        B = np.eye(2)  # Start with identity matrix as initial inverse Hessian approximation
        for _ in range(10):  # Perform multiple BFGS updates
            g_initial = grad_f(x)
            s = -np.dot(H, g_initial)  # Step using the current inverse Hessian
            x_new = x + s
            g_new = grad_f(x_new)
            y = g_new - g_initial
            
            B = bfgs_update_B(B, s, y)  # Update H_k using BFGS formula
            H = bfgs_update_H(H, s, y)  # Update H_k using BFGS formula
            x = x_new
        
        # Check if H has converged to the inverse of the true Hessian Q
        np.testing.assert_allclose(B, Q, atol=1e-3, rtol=1e-3, err_msg="BFGS B_k did not converge to true Hessian after multiple steps")
        np.testing.assert_allclose(H, Q_inv, atol=1e-3, rtol=1e-3, err_msg="BFGS H_k did not converge to true inverse Hessian after multiple steps")

# Execute the test
test_suite = unittest.TestSuite()
loader = unittest.TestLoader()
test_suite.addTest(loader.loadTestsFromTestCase(TestBFGSHessianAndInverseUpdate))
runner = unittest.TextTestRunner()
runner.run(test_suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

#### Symmetric Rank 1 (SR1) Update Method

**Description:**
The SR1 method is a quasi-Newton technique used for optimization that updates the approximate Hessian matrix or its inverse using a rank-1 update. This method is particularly effective when the exact Hessian is indefinite or changes rapidly, conditions under which traditional quasi-Newton methods like BFGS might fail or perform poorly.

**Mathematical Formulation:**
SR1 updates the inverse of the Hessian matrix using a formula that does not enforce symmetry and positive definiteness, giving it flexibility in handling diverse types of objective functions:

1. **Initialization:**
   - Start with an initial guess \( x_0 \).
   - Initialize the inverse Hessian approximation \( H_0 \), often as the identity matrix.

2. **Iterative Update:**
   - For each iteration \( k \), execute the following:
     - Compute the gradient: \( g_k = \nabla f(x_k) \).
     - Compute the search direction: \( p_k = -H_k g_k \).
     - Determine the step size \( \alpha_k \) using a line search to minimize \( f(x_k + \alpha_k p_k) \).
     - Update the position: \( x_{k+1} = x_k + \alpha_k p_k \).
     - Update the gradient: \( g_{k+1} = \nabla f(x_{k+1}) \).
     - Calculate the change vectors: \( s_k = x_{k+1} - x_k \) and \( y_k = g_{k+1} - g_k \).
     - If \( (s_k - H_k y_k)^T y_k \neq 0 \), update the inverse Hessian approximation:
       $$ H_{k+1} = H_k + \frac{(s_k - H_k y_k)(s_k - H_k y_k)^T}{(s_k - H_k y_k)^T y_k} $$

**Algorithmic Steps:**
1. Evaluate the gradient and decide the search direction.
2. Use line search to find the appropriate step size.
3. Update the position and evaluate the new gradient.
4. Adjust the inverse Hessian approximation using the SR1 update formula.
5. Check for convergence based on the gradient norm or a change in function value, and repeat the steps if necessary.

**Rationale and Use Case:**
SR1 is beneficial in situations where the Hessian has significant curvature changes that are not well captured by other quasi-Newton updates. Its flexibility can lead to superior convergence properties in non-quadratic and ill-conditioned problems.

**Heuristics and Parameter Choices:**
- The initial approximation \( H_0 \) and the line search method are crucial for the performance and stability of the SR1 method.
- It's essential to handle cases where the denominator in the update formula approaches zero, which might require skipping the update.

**Advantages and Limitations:**
- **Advantages:** Potentially more accurate Hessian approximation in dynamically changing landscapes. Less restrictive update rule can adapt better to complex objective functions.
- **Limitations:** The SR1 update can fail if the curvature condition \( (s_k - H_k y_k)^T y_k = 0 \) is met, requiring fallback strategies. It may also converge slower than BFGS in some cases.


In [46]:
def sr1_update(H, s, y, quasi_zero=1e-12):
    """ Perform the SR1 update on the approximate Hessian (or its inverse).
    
    Args:
        H (np.ndarray): Current Hessian approximation.
        s (np.ndarray): Step vector x_k+1 - x_k.
        y (np.ndarray): Change in gradients grad f(x_k+1) - grad f(x_k).
    
    Returns:
        np.ndarray: Updated Hessian approximation if curvature condition is met.
    """
    u = y - H @ s
    uT_s = np.dot(u, s)
    if np.abs(uT_s) > quasi_zero:  # avoiding division by zero or very small numbers
        H += np.outer(u, u) / uT_s
    return H


In [47]:
import numpy as np
import unittest

def sr1_update(B, s, y):
    """ Perform the SR1 update on the approximate Hessian B.
    
    Args:
        B (np.ndarray): Current Hessian approximation.
        s (np.ndarray): Step vector x_{k+1} - x_k.
        y (np.ndarray): Change in gradients grad f(x_{k+1}) - grad f(x_k).
    
    Returns:
        np.ndarray: Updated Hessian approximation.
    """
    Bs = np.dot(B, s)
    u = y - Bs
    uTs = np.dot(u.T, s)
    if np.abs(uTs) > 1e-8:  # Avoid division by zero or very small numbers
        B_next = B + np.outer(u, u) / uTs
        return B_next
    return B  # Skip the update if the denominator is too small

class TestSR1MultiStepUpdate(unittest.TestCase):
    def test_sr1_multi_step_update_quadratic(self):
        # Define parameters for a simple quadratic function f(x) = 1/2 x^T Q x - b^T x
        Q = np.array([[3, 0.5], [0.5, 2]])  # True Hessian
        b = np.array([1, 1])
        x = np.array([0.0, 0.0])  # Initial point
        grad_f = lambda x: np.dot(Q, x) - b
        
        B = np.eye(2)  # Start with identity matrix as initial Hessian approximation
        for _ in range(10):  # Perform multiple SR1 updates
            g_initial = grad_f(x)
            # Simplified step calculation: s = -B^-1 * g
            s = -np.linalg.solve(B, g_initial)
            x_new = x + s
            g_new = grad_f(x_new)
            y = g_new - g_initial
            
            B = sr1_update(B, s, y)  # Update B using SR1 formula
            x = x_new
        
        # Check if B has converged to the true Hessian Q
        np.testing.assert_allclose(B, Q, atol=1e-3, rtol=1e-3, err_msg="SR1 did not converge to true Hessian after multiple steps")

test_suite = unittest.TestSuite()
loader = unittest.TestLoader()
test_suite.addTest(loader.loadTestsFromTestCase(TestSR1MultiStepUpdate))
runner = unittest.TextTestRunner()
runner.run(test_suite)


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [48]:
def quasi_newton_method(func, grad, x0, known_solution, method='BFGS', max_iter=1000, tol=1e-6, ls_max_iter=50):
    """ Generic optimization using quasi-Newton methods BFGS or SR1.
    
    Args:
        func (callable): The function to minimize.
        grad (callable): The gradient of the function.
        x0 (np.array): Initial guess for the solution.
        known_solution (np.array): The known solution or global minimum for the function, used for calculating distance.
        method (str): Specifies the update method ('BFGS' or 'SR1').
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance.
        ls_max_iter<. Msximum number of iterations in line search
    
    Returns:
        np.array: The optimized variable values.
    """
    x = x0
    n = len(x0)
    H = np.eye(n)  # Start with the identity matrix
    overall_log = []
    for i in range(max_iter):
        g = grad(x)
        if np.linalg.norm(g) < tol:
            break
        p = -np.linalg.solve(H, g)
        alpha, ls_log = line_search(func, grad, x, p, ls_max_iter)
        #alpha = 0.5 # for now
        s = alpha * p
        x_new = x + s
        y = grad(x_new) - g

        # Update the Hessian approximation based on the method
        if method == 'BFGS':
            H = bfgs_update(H, s, y)
        elif method == 'SR1':
            H = sr1_update(H, s, y)
        else:
            raise ValueError('Invalid method: ', method)
        
        x = x_new
        overall_log.append({
            'iteration': i + 1,
            'x': x.copy(),
            'gradient_norm': np.linalg.norm(g),
            'alpha': alpha,
            'function_value': func(x),
            'method': method
        })
        if np.linalg.norm(s) < tol:  # Additional stopping condition based on position change
            break
    
    final_gradient_norm = np.linalg.norm(g)
    distance_to_solution = np.linalg.norm(x - known_solution)
    return x, i, overall_log, final_gradient_norm, distance_to_solution

### Gradient and Hessian Approximation

#### Gradient Approximation Using Central Difference

**Description:**
Gradient approximation using the central difference formula is a numerical differentiation technique used to estimate the gradient of a scalar function. This method provides a more accurate approximation than the forward or backward difference methods by taking an average of slopes on both sides of a point.

**Mathematical Formulation:**
Given a scalar function $f: \mathbb{R}^n \to \mathbb{R}$, the gradient of $f$ at a point $x \in \mathbb{R}^n$ can be approximated using the central difference formula for each component $i$ of the gradient:

$$ \frac{\partial f}{\partial x_i}(x) \approx \frac{f(x + h e_i) - f(x - h e_i)}{2h} $$

where:
- $h$ is a small step size,
- $e_i$ is the unit vector in the direction of the $i$-th coordinate.

**Algorithmic Steps:**
1. Choose a small value for $h$, typically on the order of $10^{-5}$ to $10^{-8}$, depending on the function's sensitivity and floating-point precision considerations.
2. For each dimension $i$ from 1 to $n$:
   - Compute $f(x + h e_i)$ and $f(x - h e_i)$.
   - Use the central difference formula to estimate the $i$-th component of the gradient.
3. Combine the computed components to form the gradient vector at $x$.

**Rationale and Use Case:**
Central difference is used when the function's analytical gradient is unavailable or when the function is defined implicitly by a simulation or experiment. It is widely used in optimization algorithms, especially for machine learning models trained with gradient-based methods where analytical gradients are not readily available.

**Heuristics and Parameter Choices:**
- The choice of $h$ is critical; too large a value can introduce significant approximation errors, while too small a value may lead to numerical instability due to floating-point precision errors.
- It's often useful to experiment with different values of $h$ to find a balance between accuracy and computational stability.

**Advantages and Limitations:**
- **Advantages:** Simple to implement and does not require knowledge of the function's derivative. More accurate than forward and backward difference methods.
- **Limitations:** More computationally intensive than forward or backward differences since it requires two function evaluations per gradient component. Numerical errors can still be significant if $h$ is not appropriately chosen.



In [49]:
import numpy as np

def finite_difference_gradient(func, x, h=1e-5):
    """
    Approximates the gradient of a function at a given point using the central difference formula.
    
    Args:
        func (callable): The function for which the gradient is to be approximated.
        x (np.array): The point at which the gradient is to be approximated.
        h (float): The step size for the finite difference approximation.

    Returns:
        np.array: The approximated gradient as a numpy array.
    """
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x_plus = np.array(x, dtype=float)
        x_minus = np.array(x, dtype=float)
        x_plus[i] += h  # Increment x[i] by h
        x_minus[i] -= h  # Decrement x[i] by h
        grad[i] = (func(x_plus) - func(x_minus)) / (2 * h)  # Central difference for derivative
    return grad


#### Hessian Approximation Using Finite Differences

**Description:**
Hessian approximation using finite differences is a numerical method to estimate the Hessian matrix, which is the second-order partial derivatives matrix of a scalar function. This approximation is crucial for methods that utilize curvature information to find minima or maxima of functions, especially in the absence of explicit second derivative formulas.

**Mathematical Formulation:**
Given a scalar function $f: \mathbb{R}^n \to \mathbb{R}$, the Hessian matrix $H$ at a point $x \in \mathbb{R}^n$ can be approximated using finite differences. Each element $H_{ij}$ of the Hessian matrix is approximated as:

$$ H_{ij}(x) \approx \frac{f(x + h e_i + h e_j) - f(x + h e_i) - f(x + h e_j) + f(x)}{h^2} $$

where:
- $h$ is a small step size,
- $e_i$ and $e_j$ are unit vectors in the directions of the $i$-th and $j$-th coordinates, respectively.

**Algorithmic Steps:**
1. Choose a small value for $h$, typically on the order of $10^{-5}$ to $10^{-8}$.
2. For each pair of dimensions $(i, j)$:
   - Compute the four function values needed for the finite difference approximation of the second derivative.
   - Use the above formula to estimate each element $H_{ij}$ of the Hessian matrix.
3. Assemble the computed elements into the full Hessian matrix.

**Rationale and Use Case:**
This method is particularly useful in optimization contexts where the second-order derivative (Hessian) information enhances the performance of algorithms, enabling more accurate and faster convergence to minima or maxima. It is essential for applications lacking analytical derivative expressions, such as complex engineering designs or machine learning models involving non-differentiable components.

**Heuristics and Parameter Choices:**
- The selection of $h$ is critical and should be chosen to balance the trade-off between numerical accuracy and stability. Too small a value may lead to floating-point precision issues, while too large a value can introduce significant approximation errors.
- Symmetry of the Hessian can be enforced by averaging $H_{ij}$ and $H_{ji}$.

**Advantages and Limitations:**
- **Advantages:** Allows for the use of second-order methods when derivatives are not analytically available, improving convergence properties of optimization algorithms.
- **Limitations:** Computationally expensive as it requires $O(n^2)$ function evaluations for a full Hessian matrix approximation. Numerical errors and instability can occur if $h$ is not appropriately chosen.



In [50]:
def finite_difference_hessian(func, x, h=1e-4):
    """
    Approximates the Hessian matrix of a function at a given point using finite differences.
    
    Args:
        func (callable): The function for which the Hessian is to be approximated.
        x (np.array): The point at which the Hessian is to be approximated.
        h (float): The step size for the finite difference approximation.

    Returns:
        np.array: The approximated Hessian matrix as a 2D numpy array.
    """
    n = len(x)
    hessian = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            x_ij = np.array(x, dtype=float)
            if i == j:
                # Diagonal entries
                x_ij[i] += h
                f_plus = func(x_ij)
                x_ij[i] = x[i] - h
                f_minus = func(x_ij)
                hessian[i, j] = (f_plus - 2 * func(x) + f_minus) / h**2
            else:
                # Off-diagonal entries, use central difference for mixed partial derivatives
                # f(x + h*ei + h*ej) - f(x + h*ei - h*ej) - f(x - h*ei + h*ej) + f(x - h*ei - h*ej)
                x_plus_plus = np.array(x, dtype=float)
                x_plus_minus = np.array(x, dtype=float)
                x_minus_plus = np.array(x, dtype=float)
                x_minus_minus = np.array(x, dtype=float)
                
                x_plus_plus[i] += h
                x_plus_plus[j] += h
                x_plus_minus[i] += h
                x_plus_minus[j] -= h
                x_minus_plus[i] -= h
                x_minus_plus[j] += h
                x_minus_minus[i] -= h
                x_minus_minus[j] -= h
                
                f_plus_plus = func(x_plus_plus)
                f_plus_minus = func(x_plus_minus)
                f_minus_plus = func(x_minus_plus)
                f_minus_minus = func(x_minus_minus)
                
                hessian[i, j] = (f_plus_plus - f_plus_minus - f_minus_plus + f_minus_minus) / (4 * h**2)
    return hessian


In [51]:
# Map each function to its corresponding gradient function
grad_func_map = {
    custom_function: grad_custom_function,
    safe_rosenbrock: safe_grad_rosenbrock,
}


In [52]:
# Example of defining a global map for Hessian functions
hessian_func_map = {
    custom_function: hessian_custom_function,  # Example: define `analytical_hessian_custom` appropriately
    safe_rosenbrock: hessian_rosenbrock  # Define this function if needed
}


In [53]:
def get_gradient_func(func, use_numerical_gradient=False):
    """
    Retrieves the appropriate gradient function for a given objective function, allowing for the option
    to use either the analytical or numerical gradient.

    Args:
        func (callable): The objective function for which the gradient is required. This function
                         should take a single numpy array as input and return a single value.
        use_numerical_gradient (bool): A flag indicating whether to use a numerical gradient approximation.
                                       If True, the gradient will be estimated using finite differences;
                                       otherwise, the pre-defined analytical gradient will be used.

    Returns:
        callable: A function that computes the gradient of the objective function. This returned function
                  takes a numpy array as input and returns a numpy array representing the gradient at that point.
                  
    Examples:
        # Define an objective function
        def my_function(x):
            return x[0]**2 + x[1]**2 + 3*x[0]*x[1]

        # Get the analytical gradient function
        analytical_grad = get_gradient_func(my_function, use_numerical_gradient=False)
        
        # Get the numerical gradient function
        numerical_grad = get_gradient_func(my_function, use_numerical_gradient=True)
        
        # Compute gradients
        x = np.array([1.0, 2.0])
        print("Analytical Gradient:", analytical_grad(x))
        print("Numerical Gradient:", numerical_grad(x))
    """
    if use_numerical_gradient:
        # Use the numerical gradient approximation via finite differences
        return lambda x: finite_difference_gradient(func, x)
    else:
        # Retrieve the analytical gradient from a predefined map
        try:
            return grad_func_map[func]
        except KeyError:
            raise ValueError("Analytical gradient function not defined for the provided function.")


In [54]:
def get_hessian_func(func, use_numerical_hessian=False, use_quasi_newton=False):
    """
    Retrieves the appropriate Hessian function based on the configuration.

    Args:
        func (callable): The function for which the Hessian is needed.
        use_numerical_hessian (bool): If True, use a numerical method to approximate the Hessian.
        use_quasi_newton (bool): If True, use a quasi-Newton method for the Hessian approximation.

    Returns:
        callable: A function that computes the Hessian matrix for the given function.
    """
    if use_numerical_hessian:
        # Return a lambda function that calculates the numerical Hessian
        return lambda x: finite_difference_hessian(func, x)
    elif use_quasi_newton:
        # If implementing a specific quasi-Newton method like BFGS that approximates the Hessian
        # Normally BFGS would update its Hessian approximation internally, so this might be managed differently
        return None  # Placeholder for quasi-Newton Hessian management
    else:
        # Return the analytical Hessian function mapped to `func`
        return hessian_func_map[func]  # Ensure this map is defined somewhere globally



## Experiment Configuration

We will test the optimization algorithms on two benchmark functions with the following initial points:

- Rosenbrock function: Points `[1.2, 1.2]`, `[-1.2, 1.0]`, and `[0.2, 0.8]`.
- Custom function: Points `[-0.2, 1.2]`, `[3.8, 0.1]`, and `[1.9, 0.6]`.

Each test will be run with a convergence tolerance of \(1e-6\) and a maximum of 500000 iterations, allowing us to assess the efficiency and effectiveness of each method under various conditions.


In [55]:
# Define initial points for Rosenbrock function
initial_points_rosenbrock = [np.array([1.2, 1.2]), np.array([-1.2, 1.0]), np.array([0.2, 0.8])]

# Define initial points for the custom function
initial_points_custom = [np.array([-0.2, 1.2]), np.array([3.8, 0.1]), np.array([1.9, 0.6])]


In [56]:
import pandas as pd
import numpy as np
import time

def run_optimization_test(opt_method, func, x0, known_solution, grad=None, hess=None, use_grad_approx=False, use_hess_approx=False, max_iter=1000, tol=1e-6):
    """
    Runs optimization tests using the specified optimization method, allowing for gradient and Hessian approximations.

    Args:
        opt_method (str): Specifies the optimization method variant ('FR', 'PR', 'BFGS', 'SR1', 'Newton', 'LCG').
        func (callable): The objective function to minimize.
        x0 (list of np.array): Initial points for the optimization.
        known_solution (np.array): The known global minimum of the function.
        grad (callable, optional): The exact gradient of the function, if available.
        hess (callable, optional): The exact Hessian of the function, if applicable.
        use_grad_approx (bool): Flag to use gradient approximation instead of exact gradient.
        use_hess_approx (bool): Flag to use Hessian approximation instead of exact Hessian.
        max_iter (int): Maximum number of iterations.
        tol (float): Convergence tolerance.

    Returns:
        pandas.DataFrame: DataFrame containing results from each optimization run.
    """
    #print(f"Processing opt_method={opt_method}, func={func}, x0={x0}, known_solution={known_solution}, grad={grad}, hess={hess}, use_grad_approx={use_grad_approx}, use_hess_approx={use_hess_approx}, max_iter={max_iter}, tol={tol}")
    results = []
    current_grad = get_gradient_func(func, use_numerical_gradient=use_grad_approx)
    current_hess = get_hessian_func(func, use_numerical_hessian=use_hess_approx) if opt_method in ['BFGS', 'SR1', 'Newton'] else None
    #print(f"Determined current_grad as {current_grad}, current_hess a {current_hess}")
    for initial_point in x0:
        start = time.time()

        if opt_method in ['FR', 'PR']:
            x_final, num_iterations, overall_log, final_gradient_norm, distance_to_solution = conjugate_gradient_method(
                func, current_grad, initial_point, known_solution, method=opt_method, max_iter=max_iter, tol=tol
            )
        elif opt_method in ['BFGS', 'SR1']:
            x_final, num_iterations, overall_log, final_gradient_norm, distance_to_solution = quasi_newton_method(
                func, current_grad, initial_point, known_solution, method=opt_method, max_iter=max_iter, tol=tol
            )
        elif opt_method in ['Newton']:
            x_final, num_iterations, overall_log, final_gradient_norm, distance_to_solution = newton_eigen_method(
                func, current_grad, current_hess, initial_point, known_solution, max_iter=max_iter, tol=tol
            )
        elif opt_method == 'LCG':
            x_final, num_iterations, overall_log, final_gradient_norm, distance_to_solution = linear_conjugate_gradient(
                func, current_grad, initial_point, known_solution, max_iter=max_iter, tol=tol
            )
        else:
            raise ValueError(f'Invalid optimization method: {opt_method}')

        stop = time.time()
        results.append({
            'Starting Point': np.array_str(initial_point),
            'Known Solution': np.array_str(known_solution),
            'Calculated Solution': np.array_str(x_final),
            'Distance to Solution': distance_to_solution,
            'Final Gradient Norm': final_gradient_norm,
            'Number of Iterations': num_iterations,
            'Execution Time (s)': stop - start
        })

    return pd.DataFrame(results)

# Define finite difference methods if not already defined
def finite_difference_gradient(func, x, epsilon=1e-8):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x1 = np.array(x, dtype=float)
        x2 = np.array(x, dtype=float)
        x1[i] += epsilon
        x2[i] -= epsilon
        grad[i] = (func(x1) - func(x2)) / (2 * epsilon)
    return grad

def finite_difference_hessian(func, x, epsilon=1e-6):
    n = len(x)
    hessian = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            x_ijp = np.array(x, dtype=float)
            x_ijm = np.array(x, dtype=float)
            x_ijp[i] += epsilon
            x_ijp[j] += epsilon
            x_ijm[i] -= epsilon
            x_ijm[j] -= epsilon
            if i == j:
                hessian[i, j] = (func(x_ijp) - 2 * func(x) + func(x_ijm)) / epsilon**2
            else:
                f_ip_jp = func(x_ijp)
                x_ijp[j] -= 2 * epsilon
                f_ip_jm = func(x_ijp)
                x_ijm[i] += 2 * epsilon
                f_im_jp = func(x_ijm)
                x_ijm[j] += 2 * epsilon
                f_im_jm = func(x_ijm)
                hessian[i, j] = (f_ip_jp - f_ip_jm - f_im_jp + f_im_jm) / (4 * epsilon**2)
    return hessian

# Example usage
#initial_points = [np.array([-1.2, 1.0]), np.array([1.0, 1.0]), np.array([-1.5, 1.5])]
#initial_points = [np.array([-1.2, 1.0])]
#known_solution = np.array([1, 1])

#results_df = run_optimization_test(
#    opt_method='BFGS',
#    func=safe_rosenbrock,
#    x0=initial_points,
#    known_solution=known_solution,
#    use_grad_approx=False,
#    use_hess_approx=False
#)
#print(results_df)

## Optimization Results

Below are the detailed results from each optimization run:


In [57]:
known_solution_rosenbrock = np.array([1., 1.])
known_solution_custom = np.array([4., 0.]) # or (0,1) //FIXME

# Run tests and display results

results_df_rosenbrock_nm = run_optimization_test(
    opt_method='Newton',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Newton):")
display(results_df_rosenbrock_nm)

results_df_custom_nm = run_optimization_test(
    opt_method='Newton',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Newton):")
display(results_df_custom_nm)

results_df_rosenbrock_lcg = run_optimization_test(
    opt_method='LCG',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Linear Conjugate Gradient):")
display(results_df_rosenbrock_lcg)

results_df_custom_lcg = run_optimization_test(
    opt_method='LCG',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Linear Conjugate Gradient):")
display(results_df_custom_lcg)

results_df_rosenbrock_fr = run_optimization_test(
    opt_method='FR',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Fletcher-Reeves):")
display(results_df_rosenbrock_fr)

results_df_custom_fr = run_optimization_test(
    opt_method='FR',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Fletcher-Reeves):")
display(results_df_custom_fr)

# ATTENTION! Running _long_: 4:50 for 10000 iters
results_df_rosenbrock_pr = run_optimization_test(
    opt_method='PR',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
   use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Polak-Ribiere):")
display(results_df_rosenbrock_pr)

# ATTENTION! Running _long_
results_df_custom_pr = run_optimization_test(
    opt_method='PR',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Polak-Ribiere):")
display(results_df_custom_pr)

# ATTENTION! Running _long_: 4:50 for 10000 iters
results_df_rosenbrock_bfgs = run_optimization_test(
    opt_method='BFGS',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (BFGS):")
display(results_df_rosenbrock_bfgs)

# ATTENTION! Running _long_
results_df_custom_bfgs = run_optimization_test(
    opt_method='BFGS',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (BFGS):")
display(results_df_custom_bfgs)

results_df_rosenbrock_sr1 = run_optimization_test(
    opt_method='SR1',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (SR1):")
display(results_df_rosenbrock_bfgs)

results_df_custom_sr1 = run_optimization_test(
    opt_method='SR1',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=False,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (SR1):")
display(results_df_custom_bfgs)


'Results for Rosenbrock Function (Newton):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[1.2 1.2],[1. 1.],[1. 1.],1.339488e-10,2.056839e-10,8,0.002193
1,[-1.2 1. ],[1. 1.],[0.99999999 0.99999999],1.290956e-08,7.557077e-09,20,0.005942
2,[0.2 0.8],[1. 1.],[1.00000001 1.00000001],1.179722e-08,1.877807e-08,9,0.009507


'Results for Custom Function (Newton):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[-1.86734201e-10 1.00000000e+00],4.123106e+00,5.476743e-08,8,0.002190
1,[3.8 0.1],[4. 0.],[ 4.00000000e+00 -2.46043639e-11],1.276079e-09,1.157476e-07,8,0.003394
2,[1.9 0.6],[4. 0.],[ 4.0000000e+00 -5.3447754e-13],2.035943e-10,2.164925e-09,11,0.008478


'Results for Rosenbrock Function (Linear Conjugate Gradient):'

...

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[-1.33701476e-05 1.00074215e+00],4.123299,0.006464,9999,13.593201
1,[3.8 0.1],[4. 0.],[3.79773013e+00 9.50859422e-05],0.202270,0.101289,9999,15.458590
2,[1.9 0.6],[4. 0.],[4.00036318e+00 2.01133823e-07],0.000363,0.001714,1325,2.424134


'Results for Rosenbrock Function (SR1):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[1.2 1.2],[1. 1.],[1.11119841 1.23515821],0.260124,0.091774,9999,72.247314
1,[-1.2 1. ],[1. 1.],[1.42495122 2.03124789],1.115372,0.437079,9999,74.640278
2,[0.2 0.8],[1. 1.],[1.82763086 3.34307014],2.484945,0.641518,9999,76.502512


'Results for Custom Function (SR1):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[-1.33701476e-05 1.00074215e+00],4.123299,0.006464,9999,13.593201
1,[3.8 0.1],[4. 0.],[3.79773013e+00 9.50859422e-05],0.202270,0.101289,9999,15.458590
2,[1.9 0.6],[4. 0.],[4.00036318e+00 2.01133823e-07],0.000363,0.001714,1325,2.424134


With gradient and Hessian approximation (if applicable):

In [ ]:
known_solution_rosenbrock = np.array([1., 1.])
known_solution_custom = np.array([4., 0.]) # or (0,1) //FIXME

# Run tests and display results

results_df_rosenbrock_nm = run_optimization_test(
    opt_method='Newton',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Newton):")
display(results_df_rosenbrock_nm)

results_df_custom_nm = run_optimization_test(
    opt_method='Newton',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Newton):")
display(results_df_custom_nm)

results_df_rosenbrock_lcg = run_optimization_test(
    opt_method='LCG',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Linear Conjugate Gradient):")
display(results_df_rosenbrock_lcg)

results_df_custom_lcg = run_optimization_test(
    opt_method='LCG',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Linear Conjugate Gradient):")
display(results_df_custom_lcg)

results_df_rosenbrock_fr = run_optimization_test(
    opt_method='FR',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Fletcher-Reeves):")
display(results_df_rosenbrock_fr)

results_df_custom_fr = run_optimization_test(
    opt_method='FR',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Fletcher-Reeves):")
display(results_df_custom_fr)

# ATTENTION! Running _long_: 4:50 for 10000 iters
results_df_rosenbrock_pr = run_optimization_test(
    opt_method='PR',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
   use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (Polak-Ribiere):")
display(results_df_rosenbrock_pr)

# ATTENTION! Running _long_
results_df_custom_pr = run_optimization_test(
    opt_method='PR',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (Polak-Ribiere):")
display(results_df_custom_pr)

# ATTENTION! Running _long_: 4:50 for 10000 iters
results_df_rosenbrock_bfgs = run_optimization_test(
    opt_method='BFGS',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (BFGS):")
display(results_df_rosenbrock_bfgs)

# ATTENTION! Running _long_
results_df_custom_bfgs = run_optimization_test(
    opt_method='BFGS',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (BFGS):")
display(results_df_custom_bfgs)

results_df_rosenbrock_sr1 = run_optimization_test(
    opt_method='SR1',
    func=safe_rosenbrock,
    x0=initial_points_rosenbrock,
    known_solution=known_solution_rosenbrock,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Rosenbrock Function (SR1):")
display(results_df_rosenbrock_bfgs)

results_df_custom_sr1 = run_optimization_test(
    opt_method='SR1',
    func=custom_function,
    x0=initial_points_custom,
    known_solution=known_solution_custom,
    use_grad_approx=True,
    use_hess_approx=False,
    max_iter=10000,
    tol=1e-6
)
display("Results for Custom Function (SR1):")
display(results_df_custom_bfgs)


'Results for Rosenbrock Function (Newton):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[1.2 1.2],[1. 1.],[1. 1.],1.344800e-10,2.078589e-10,8,0.004013
1,[-1.2 1. ],[1. 1.],[0.99999999 0.99999999],1.291168e-08,7.573559e-09,20,0.011561
2,[0.2 0.8],[1. 1.],[1.00000001 1.00000001],1.180466e-08,1.880027e-08,9,0.019549


'Results for Custom Function (Newton):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[-1.86735967e-10 1.00000000e+00],4.123106e+00,5.476784e-08,8,0.007184
1,[3.8 0.1],[4. 0.],[ 4.00000000e+00 -2.46022985e-11],1.276370e-09,1.157371e-07,8,0.007957
2,[1.9 0.6],[4. 0.],[ 4.00000000e+00 -4.13355549e-12],1.549486e-09,1.679268e-08,11,0.015792


'Results for Rosenbrock Function (Linear Conjugate Gradient):'

...

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[8.25406356e-10 9.99999879e-01],4.123106e+00,9.680868e-07,653,0.950621
1,[3.8 0.1],[4. 0.],[ 3.99999994e+00 -1.79507442e-10],5.596716e-08,9.754174e-07,469,1.212040
2,[1.9 0.6],[4. 0.],[ 4.00000009e+00 -2.26491052e-10],8.738977e-08,9.152122e-07,452,0.788856


'Results for Rosenbrock Function (Fletcher-Reeves):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[1.2 1.2],[1. 1.],[1.00000005 1.0000001 ],1.140972e-07,9.512816e-07,296,2.757622
1,[-1.2 1. ],[1. 1.],[1.00000105 1.00000209],2.338675e-06,9.366804e-07,650,4.775309
2,[0.2 0.8],[1. 1.],[0.99999904 0.99999808],2.143373e-06,8.684976e-07,536,4.039246


'Results for Custom Function (Fletcher-Reeves):'

,Starting Point,Known Solution,Calculated Solution,Distance to Solution,Final Gradient Norm,Number of Iterations,Execution Time (s)
0,[-0.2 1.2],[4. 0.],[1.88043448e-08 9.99999942e-01],4.123106e+00,9.680868e-07,653,1.301962
1,[3.8 0.1],[4. 0.],[3.99999994e+00 2.32950144e-10],5.584727e-08,9.754174e-07,469,0.897800
2,[1.9 0.6],[4. 0.],[4.00000009e+00 1.83707111e-10],8.753636e-08,9.152122e-07,452,0.769504


### Results Analysis

The results from the optimization tests on both the Rosenbrock and Custom functions illustrate key differences in the convergence behavior of the different methods. For example, the Fletcher-Reeves method showed faster convergence on the Rosenbrock function from closer initial points to the global minimum, while Polak-Ribiere was more robust to poor initial conditions on the Custom function.

Even more surprising, some methods reached their optimization targets in reasonably few iterations where others required extraordinary amounts of runtime and resources.

These findings suggest that the choice of method and starting point can significantly impact the efficiency and success of optimization, especially in complex or ill-conditioned problem spaces. The choice of whether to approximate derivatives seems to have had comparably little impact, though.


# Bonus Assignment
<h3>Goal: Outperform NM with QN</h3>

## Our Function
$$
f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2
$$

## Our Implementation

In [ ]:
import numpy as np

def bonus_f(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

def bonus_grad_f(x):
    df_dx = 2 * (x[0]**2 + x[1] - 11) * 2 * x[0] + 2 * (x[0] + x[1]**2 - 7)
    df_dy = 2 * (x[0]**2 + x[1] - 11) * 1 + 2 * (x[0] + x[1]**2 - 7) * 2 * x[1]
    return np.array([df_dx, df_dy])

def bonus_hessian_f(x):
    d2f_dx2 = 2 * (2 * x[0]) * (2 * x[0]**2 + 2 * x[1] - 11) + 2 * 2 * (x[0] + x[1]**2 - 7)
    d2f_dy2 = 2 * (2 * x[0]**2 + 2 * x[1] - 11) * (1) + 2 * 2 * (x[0] + x[1]**2 - 7) * (2 * x[1])
    d2f_dxdy = 2 * (2 * x[0]) * (1) + 2 * 2 * (x[0] + x[1]**2 - 7) * (2 * x[1])
    return np.array([[d2f_dx2, d2f_dxdy], [d2f_dxdy, d2f_dy2]])

def bonus_backtracking_line_search(f, x, grad, p, alpha=0.3, beta=0.5):
    t = 1.0
    while f(x + t * p) > f(x) + alpha * t * np.dot(grad, p):
        t *= beta
    return t

def bonus_newton_method(f, grad, hessian, x0, max_iter=1000, tol=1e-6):
    x = x0.copy()
    for i in range(max_iter):
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < tol:
            break
        hess_x = hessian(x)
        try:
            p = np.linalg.solve(hess_x, -grad_x)
        except np.linalg.LinAlgError:
            break  # In case Hessian is singular, break
        t = bonus_backtracking_line_search(f, x, grad_x, p)
        x += t * p
    return x, np.linalg.norm(grad_x), i

def bonus_bfgs_update(H, s, y):
    ys = np.dot(y, s)
    if ys < 1e-10:  # Prevent division by zero or very small values
        return H
    rho = 1.0 / ys
    I = np.eye(len(H))
    V = I - rho * np.outer(s, y)
    H = V.T @ H @ V + rho * np.outer(s, s)
    return H

def bonus_quasi_newton_method(f, grad, x0, max_iter=1000, tol=1e-6):
    x = x0.copy()
    n = len(x)
    H = np.eye(n)
    for i in range(max_iter):
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < tol:
            break
        p = -np.dot(H, grad_x)
        t = bonus_backtracking_line_search(f, x, grad_x, p)
        s = t * p
        x_next = x + s
        y = grad(x_next) - grad_x
        if np.dot(y, s) > 1e-10:
            H = bfgs_update(H, s, y)
        x = x_next
    return x, np.linalg.norm(grad(x)), i


## Our Run

In [ ]:
x0_list = [(1.0, 1.0), (1.2, 1.2), (-1.2, 1), (0.2, 0.8)]
for i, x0 in enumerate(x0_list):
    print(f"Starting point {i+1}: {x0}")
    x, grad_norm, num_iter = bonus_newton_method(f, bonus_grad_f, bonus_hessian_f, np.array(x0))
    print(f"Newton Method: Iterations: {num_iter}, Final iterate: {x}, Gradient norm: {grad_norm}")

    x, grad_norm, num_iter = bonus_quasi_newton_method(f, bonus_grad_f, np.array(x0))
    print(f"Quasi-Newton Method: Iterations: {num_iter}, Final iterate: {x}, Gradient norm: {grad_norm}\n")

## Our Interpretation

Based on the optimization results for different starting points, the performance of Newton's method and Quasi-Newton method can be compared.

Starting with the initial point (1.0, 1.0), Newton's method required 999 iterations to (still not) converge to a final iterate of (-0.94897959, -2.45918367) with a gradient norm of 44.335083754394404, while the Quasi-Newton method achieved convergence in just 15 iterations, reaching a final iterate of (3.0, 2.00000001) with a much smaller gradient norm of 4.390315402132542e-07.

Similarly, at starting point (1.2, 1.2), Newton's method reached a final iterate of converged in 6 iterations to (3.0, 2.0) with a negligible gradient norm of 2.6752441484954133e-11, whereas the Quasi-Newton method required 17 iterations to converge to (3.0, 1.99999999) with a slightly higher gradient norm of 7.085236158428695e-07.

For the starting point (-1.2, 1.0), Newton's method again needed 999 iterations to (still not) converge to (-1.2, 1.0) with a gradient norm of 53.11210543746123, while the Quasi-Newton method achieved convergence in just 11 iterations, reaching (-2.80511809, 3.13131251) with a gradient norm of 3.8978054414003885e-07.

Finally, at (0.2, 0.8), Newton's method required 999 iterations to (still not converge and) reach (-0.59097274, -1.66551889) with a gradient norm of 20.859414016004283, whereas the Quasi-Newton method converged in 20 iterations to (3.00000001, 1.99999998) with a similar gradient norm of 6.631955937915146e-07.

Overall, the Quasi-Newton method dominantly outperformed Newton's method in terms of the number of iterations required for convergence and the achieved gradient norm across different starting points.
